In [1]:
import pandas as pd
import os
from prophet import Prophet
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

C:\Users\82107\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 데이터 불러오기

In [2]:
os.chdir('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation')

In [3]:
data = pd.read_csv('normal_simulation.csv') 

In [4]:
# datetime 형태로 변경
data['update_time'] = pd.to_datetime(data['update_time'])

In [5]:
data.shape

(1350, 2001)

In [6]:
data.head()

,update_time,heart_rate1,event1,heart_rate2,event2,heart_rate3,event3,heart_rate4,event4,heart_rate5,...,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,heart_rate1000,event1000
0,2023-10-01 00:00:00,72,0,101,1,77,0,77,0,95,...,83,0,82,0,64,0,66,0,84,0
1,2023-10-01 00:10:00,69,0,70,0,99,0,92,0,95,...,90,0,82,0,87,0,89,0,87,0
2,2023-10-01 00:20:00,92,0,90,0,100,0,85,0,81,...,87,0,88,0,90,0,78,0,87,0
3,2023-10-01 00:30:00,84,0,74,0,86,0,81,0,78,...,85,0,76,0,82,0,65,0,90,0
4,2023-10-01 00:40:00,85,0,75,0,67,0,83,0,103,...,71,0,84,0,81,0,85,0,73,0


In [7]:
# 최종 결과 데이터 만들기
pred = pd.DataFrame()

In [8]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df = data[['update_time', f'heart_rate{i}', f'event{i}', ]]
    # columns 명 변경
    df.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df[:1008]
    test = df[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred[f'heart_rate{i}'] = performance[['y']]
    pred[f'event{i}'] = test[[f'event{i}']]
    pred[f'pred_y{i}'] = performance[f'pred_y{i}']
    

23:00:30 - cmdstanpy - INFO - Chain [1] start processing
23:00:30 - cmdstanpy - INFO - Chain [1] done processing
23:00:31 - cmdstanpy - INFO - Chain [1] start processing
23:00:31 - cmdstanpy - INFO - Chain [1] done processing
23:00:32 - cmdstanpy - INFO - Chain [1] start processing
23:00:32 - cmdstanpy - INFO - Chain [1] done processing
23:00:32 - cmdstanpy - INFO - Chain [1] start processing
23:00:32 - cmdstanpy - INFO - Chain [1] done processing
23:00:33 - cmdstanpy - INFO - Chain [1] start processing
23:00:33 - cmdstanpy - INFO - Chain [1] done processing
23:00:34 - cmdstanpy - INFO - Chain [1] start processing
23:00:34 - cmdstanpy - INFO - Chain [1] done processing
23:00:35 - cmdstanpy - INFO - Chain [1] start processing
23:00:35 - cmdstanpy - INFO - Chain [1] done processing
23:00:36 - cmdstanpy - INFO - Chain [1] start processing
23:00:36 - cmdstanpy - INFO - Chain [1] done processing
23:00:36 - cmdstanpy - INFO - Chain [1] start processing
23:00:37 - cmdstanpy - INFO - Chain [1]

In [9]:
pred

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,73,0,0,86,0,0,76,0,0,63,...,0,80,0,0,75,0,0,73,0,0
1,83,0,0,82,0,0,71,0,0,75,...,0,78,0,0,91,0,0,88,0,0
2,90,0,0,62,0,0,70,0,0,57,...,0,63,0,0,66,0,0,74,0,0
3,81,0,0,66,0,0,72,0,0,68,...,0,76,0,0,85,0,0,76,0,0
4,96,0,1,70,0,0,73,0,0,85,...,0,86,0,0,87,0,0,75,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,83,0,0,95,0,0,77,0,0,63,...,1,83,0,0,86,0,0,88,0,0
338,74,0,0,79,0,0,87,0,0,76,...,0,91,0,0,75,0,0,70,0,0
339,83,0,0,89,0,0,100,0,1,92,...,0,66,0,0,89,0,0,84,0,0
340,110,1,1,81,0,0,82,0,0,72,...,0,71,0,0,70,0,0,65,0,0


In [12]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [15]:
predict = pd.concat([test1, pred], axis = 1)

In [16]:
predict

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2023-10-08 00:00:00,73,0,0,86,0,0,76,0,0,...,0,80,0,0,75,0,0,73,0,0
1,2023-10-08 00:10:00,83,0,0,82,0,0,71,0,0,...,0,78,0,0,91,0,0,88,0,0
2,2023-10-08 00:20:00,90,0,0,62,0,0,70,0,0,...,0,63,0,0,66,0,0,74,0,0
3,2023-10-08 00:30:00,81,0,0,66,0,0,72,0,0,...,0,76,0,0,85,0,0,76,0,0
4,2023-10-08 00:40:00,96,0,1,70,0,0,73,0,0,...,0,86,0,0,87,0,0,75,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2023-10-10 08:10:00,83,0,0,95,0,0,77,0,0,...,1,83,0,0,86,0,0,88,0,0
338,2023-10-10 08:20:00,74,0,0,79,0,0,87,0,0,...,0,91,0,0,75,0,0,70,0,0
339,2023-10-10 08:30:00,83,0,0,89,0,0,100,0,1,...,0,66,0,0,89,0,0,84,0,0
340,2023-10-10 08:40:00,110,1,1,81,0,0,82,0,0,...,0,71,0,0,70,0,0,65,0,0


In [19]:
predict.to_csv('all_normal_Prophet_predict.csv')